In [38]:
!pip install prettytable
!pip install jupyterthemes
!pip install pandas

# Библиотечный код

In [50]:
from datetime import datetime, timedelta
from prettytable import PrettyTable
import pandas as pd

from IPython.display import display, HTML






class PlantType:
    def __init__(self, name, germination_time, growth_time, seed_treatment_required=False, seed_treatment_time=0, seeds_per_box=0):
        self.name = name
        self.germination_time = germination_time
        self.growth_time = growth_time
        self.seed_treatment_required = seed_treatment_required
        self.seed_treatment_time = seed_treatment_time
        self.seeds_per_box = seeds_per_box

class PlantBox:
    def __init__(self, plant_type, start_date, this_date):
        self.end_germination_date = start_date + timedelta(days=plant_type.germination_time)
        self.end_growth_date = self.end_germination_date + timedelta(days=plant_type.growth_time)
        self.plant_type = plant_type
        self.start_date = start_date
        self.status = (
            "ready" if this_date >= self.end_growth_date else "growth" if this_date >= self.end_germination_date else "germination"
        )

        
def add_to_field_state(schedule, plant_type, start_date):
    grow_date = start_date + timedelta(days=plant_type.germination_time)

    for add in range(0, plant_type.germination_time + plant_type.growth_time):
        this_date = start_date + timedelta(days=add)
        new_box = PlantBox(plant_type, start_date, this_date)
        schedule.setdefault(this_date, []).append(new_box)
        
def add_to_field_state_as_grow(schedule, plant_type, grow_date):
    start_date = grow_date - timedelta(days=plant_type.germination_time + plant_type.growth_time)

    for add in range(0, plant_type.germination_time + plant_type.growth_time):
        this_date = start_date + timedelta(days=add)
        new_box = PlantBox(plant_type, start_date, this_date)
        schedule.setdefault(this_date, []).append(new_box)
        
class Order:
    def __init__(self, delivery_date, plants):
        self.delivery_date = delivery_date
        self.plants = plants

    def add_plant(self, plant_type, quantity):
        if plant_type in self.plants:
            self.plants[plant_type] += quantity
        else:
            self.plants[plant_type] = quantity

    def remove_plant(self, plant_type, quantity):
        if plant_type in self.plants:
            self.plants[plant_type] -= quantity
            if self.plants[plant_type] <= 0:
                del self.plants[plant_type]
                

def create_optimized_schedule(start_date, plant_types, max_slots, total_days, stack, target_ratios):
    plant_boxes = []
    schedule = {}

    base_date = start_date  # Используем фиксированную дату начала

    # Находим тип с наименьшим временем от посадки до готовности
    fastest_type = min(plant_types, key=lambda x: x.germination_time + x.growth_time)


    def can_grow_stack(state):
        count_of_growth = sum(1 for box in state if box.status == "growth")
        return count_of_growth <= max_slots - stack

    def seed_stack(day, plant_type):
        for _ in range(stack):
            add_to_field_state(schedule, plant_type, day)
            
    def find_plant_type_to_balance(schedule, target_ratios):
        current_ratios = {}
    
        for day_schedule in schedule.values():
            for box in day_schedule:
                if box.status == "growth":
                    current_ratios[box.plant_type] = current_ratios.get(box.plant_type, 0) + 1
    
        needed_ratios = {plant_type: target_ratios.get(plant_type, 0) - current_ratios.get(plant_type, 0) for plant_type in target_ratios}
    
        # Находим тип растения, которого не хватает для выравнивания баланса
        needed_plant_type = max(needed_ratios, key=needed_ratios.get)
    
        return needed_plant_type
        

    for day_offset in range(fastest_type.germination_time, total_days):
        seed_type = fastest_type
        this_day = base_date + timedelta(days=day_offset)
        schedule.setdefault(this_day, [])

        while can_grow_stack(schedule[this_day]):
            balance_type = find_plant_type_to_balance(schedule, target_ratios)
            seed_stack(this_day - timedelta(days=balance_type.germination_time), balance_type)

    return schedule

def create_schedule_from_orders(orders):
    schedule = {}

    def seed_stack(day, plant_type, num):
        for _ in range(num):
            add_to_field_state_as_grow(schedule, plant_type, day)

    for order in orders:
        end_date = order.delivery_date
        for plant, quantity in order.plants.items():
            seed_stack(end_date, plant, quantity)

    return schedule

def print_schedule_states(schedule):
    dates = sorted(schedule.keys())

    for date in dates:
        state = schedule[date]
        print(f"Дата: {date.strftime('%d.%m.%Y')}")
        for box in state:
            print(f"  Растение: {box.plant_type.name}, Состояние: {box.status}")
        print()
        
# def print_planting_schedule(schedule):
#     dates = sorted(schedule.keys())

#     table = PrettyTable()
#     table.field_names = ["Дата", "Растение", "Боксов", "Требуется предобработка", "Грамм семян", "Время предобработки", "Окончание прорастания", "Окончание роста"]

#     for date in dates:
#         state = schedule[date]

#         # Словарь для хранения информации о посаженных растениях в текущий день
#         planting_info = {}

#         for box in state:
#             if box.start_date == date:
#                 plant_name = box.plant_type.name
#                 planting_info[plant_name] = planting_info.get(plant_name, {"count": 0, "treatment_required": False})
#                 planting_info[plant_name]["count"] += 1
#                 if box.plant_type.seed_treatment_required:
#                     planting_info[plant_name]["treatment_required"] = True

#         for plant_name, info in planting_info.items():
#             count = info["count"]
#             treatment_required = info["treatment_required"]

#             # Берем значение семян и время предобработки из первого бокса данного растения
#             seeds_per_box = next((box.plant_type.seeds_per_box for box in state if box.start_date == date and box.plant_type.name == plant_name), 0)
#             treatment_time = next((box.plant_type.seed_treatment_time for box in state if box.start_date == date and box.plant_type.name == plant_name), 0)
            

#             table.add_row([date.strftime('%d.%m.%Y'), plant_name, count, "Да" if treatment_required else "Нет", count * seeds_per_box, treatment_time, box.end_germination_date.strftime('%d.%m.%Y'), box.end_growth_date.strftime('%d.%m.%Y') ])
            

#     print(table)

def custom_print_data_frame(df):
    custom_styling = [
        {
            "selector": "th",
            "props": [
                ("background", "#00abe7"),
                ("color", "white"),
                ("font-family", "tahoma"),
                ("text-align", "center"),
                ("font-size", "15px"),
            ],
        },
        {
            "selector": "td",
            "props": [
                ("font-family", "tahoma"),
                ("color", "black"),
                ("text-align", "left"),
                ("font-size", "15px"),
            ],
        },
        {
            "selector": "tr:nth-of-type(odd)",
            "props": [
                ("background", "white"),
            ],
        },
        {"selector": "tr:nth-of-type(even)", "props": [("background", "#e8e6e6")]},
        {"selector": "tr:hover", "props": [("background-color", "#bfeaf9")]},
        {"selector": "td:hover", "props": [("background-color", "#7fd5f3")]},
    ]
    s1 = df.style
    s1.set_table_styles(custom_styling)
    s1.hide(axis="index")
    
    # NOTE: Adding div Style with overflow!
    display(HTML("<div style='width: 100%; overflow: auto;'>" +
                 s1.to_html() +
                 "</div>"))

def print_planting_schedule(schedule):
    pd.options.display.max_columns=300
    dates = sorted(schedule.keys())

    data = {
        "Дата": [],
        "Растение": [],
        "Боксов": [],
        "Требуется предобработка": [],
        "Грамм семян": [],
        "Время предобработки": [],
        "Окончание прорастания": [],
        "Окончание роста": []
    }

    for date in dates:
        state = schedule[date]

        # Словарь для хранения информации о посаженных растениях в текущий день
        planting_info = {}

        for box in state:
            if box.start_date == date:
                plant_name = box.plant_type.name
                planting_info[plant_name] = planting_info.get(plant_name, {"count": 0, "treatment_required": False})
                planting_info[plant_name]["count"] += 1
                if box.plant_type.seed_treatment_required:
                    planting_info[plant_name]["treatment_required"] = True

        for plant_name, info in planting_info.items():
            count = info["count"]
            treatment_required = info["treatment_required"]

            # Берем значение семян и время предобработки из первого бокса данного растения
            seeds_per_box = next((box.plant_type.seeds_per_box for box in state if box.start_date == date and box.plant_type.name == plant_name), 0)
            treatment_time = next((box.plant_type.seed_treatment_time for box in state if box.start_date == date and box.plant_type.name == plant_name), 0)
            
            end_germination_date = next((box.end_germination_date for box in state if box.start_date == date and box.plant_type.name == plant_name), None)
            end_growth_date = next((box.end_growth_date for box in state if box.start_date == date and box.plant_type.name == plant_name), None)
            
            data["Дата"].append(date.strftime('%d.%m.%Y'))
            data["Растение"].append(plant_name)
            data["Боксов"].append(count)
            data["Требуется предобработка"].append("Да" if treatment_required else "Нет")
            data["Грамм семян"].append(count * seeds_per_box)
            data["Время предобработки"].append(treatment_time)
            data["Окончание прорастания"].append(end_germination_date.strftime('%d.%m.%Y') if end_germination_date else "")
            data["Окончание роста"].append(end_growth_date.strftime('%d.%m.%Y') if end_growth_date else "")

    df = pd.DataFrame(data)
    
    
    custom_print_data_frame(df)

# Инициализация типов растений

In [51]:
peas_type = PlantType("Горох", germination_time=3, growth_time=8, seed_treatment_required=True, seed_treatment_time=1, seeds_per_box=35)
sunflower_type = PlantType("Подсолнечник", germination_time=3, growth_time=6, seed_treatment_required=True, seed_treatment_time=2, seeds_per_box=20)
red_coral_type = PlantType("Редис Ред Корал", germination_time=3, growth_time=5, seed_treatment_required=False, seeds_per_box=5)
sango_type = PlantType("Редис Санго", germination_time=3, growth_time=5, seed_treatment_required=False, seeds_per_box=6)
kress_salat_type = PlantType("Кресс-салат", germination_time=3, growth_time=6, seed_treatment_required=False, seeds_per_box=6)
shavel_type = PlantType("Щавель", germination_time=10, growth_time=15, seed_treatment_required=False, seeds_per_box=0.15)
kinza_type = PlantType("Кинза", germination_time=7, growth_time=9, seed_treatment_required=True, seed_treatment_time=1, seeds_per_box=15)

# Создание оптимизированного рассписания

In [52]:
start_date = datetime(2023, 12, 27)
plant_types = [peas_type, sunflower_type, red_coral_type, sango_type, kress_salat_type ]
max_slots = 4*7*3
total_days = 30
stack = 4
# Отношение боксов, которое мы хотим достичь
target_ratios = {peas_type: 5, sunflower_type: 3, red_coral_type: 2, sango_type:2, kress_salat_type:2}

schedule = create_optimized_schedule(start_date, plant_types, max_slots, total_days, stack, target_ratios)

print_planting_schedule(schedule)

Дата,Растение,Боксов,Требуется предобработка,Грамм семян,Время предобработки,Окончание прорастания,Окончание роста
27.12.2023,Горох,12,Да,420,1,30.12.2023,07.01.2024
27.12.2023,Подсолнечник,16,Да,320,2,30.12.2023,05.01.2024
27.12.2023,Редис Ред Корал,20,Нет,100,0,30.12.2023,04.01.2024
27.12.2023,Редис Санго,20,Нет,120,0,30.12.2023,04.01.2024
27.12.2023,Кресс-салат,16,Нет,96,0,30.12.2023,05.01.2024
01.01.2024,Горох,8,Да,280,1,04.01.2024,12.01.2024
01.01.2024,Подсолнечник,8,Да,160,2,04.01.2024,10.01.2024
01.01.2024,Кресс-салат,8,Нет,48,0,04.01.2024,10.01.2024
01.01.2024,Редис Ред Корал,8,Нет,40,0,04.01.2024,09.01.2024
01.01.2024,Редис Санго,8,Нет,48,0,04.01.2024,09.01.2024


# Создаем рассписание от заказов

In [53]:
order_plants = {peas_type: 10, shavel_type: 10, kinza_type: 10}

order1 = Order(datetime(2024, 2, 22), order_plants)

schedule = create_schedule_from_orders([order1])

print_planting_schedule(schedule)

Дата,Растение,Боксов,Требуется предобработка,Грамм семян,Время предобработки,Окончание прорастания,Окончание роста
28.01.2024,Щавель,10,Нет,1.500000,0,07.02.2024,22.02.2024
06.02.2024,Кинза,10,Да,150.000000,1,13.02.2024,22.02.2024
11.02.2024,Горох,10,Да,350.000000,1,14.02.2024,22.02.2024
